# Notebook de descarga de información

## Información de datos abiertos

### `Explicación breve del conjunto de datos "Copiar lo que dice el conjunto de datos"`

## Importar librerias

In [17]:
# librerias base
import math
import os
import shutil 
import sys

#librerias externas
import pandas as pd 
import numpy as np
from sodapy import Socrata


# librerias propias
# Carpeta del proyecto
dir_proyecto, _ = os.path.split(os.getcwd())
sys.path.append(dir_proyecto)
#from SCRIPTS.funciones import *


In [28]:


def leer_parquet(carpeta,extension='parquet.gzip'):
    """
    

    Parameters
    ----------
    carpeta : TYPE
        DESCRIPTION.
    extension : TYPE, optional
        DESCRIPTION. The default is 'parquet.gzip'.

    Returns
    -------
    TYPE
        DESCRIPTION.

    """
    archivos_parquet = [f for f in os.listdir(f'{carpeta}') if extension in f]
    data_frames = []
    for archivo in archivos_parquet:
        data_frames.append(pd.read_parquet(os.path.join(carpeta,archivo)))
    return pd.concat(data_frames)

def dimensionar_conjunto(id_conjunto, **kwargs):
    """
    

    Parameters
    ----------
    id_conjunto : TYPE
        DESCRIPTION.

    Returns
    -------
    TYPE
        DESCRIPTION.

    """
    where = kwargs.get('where', '')
    client = cliente_socrata()
    conteo = client.get(id_conjunto, select = "count(1)", where = where)
    results_df = pd.DataFrame.from_records(conteo)
    return int(results_df.iloc[0,0])


def cliente_socrata(domain="www.datos.gov.co",token='grLe4ysf5HvZXuFKeZJlGZL7b'):
    """
    

    Parameters
    ----------
    domain : TYPE, optional
        DESCRIPTION. The default is "www.datos.gov.co".
    token : TYPE, optional
        DESCRIPTION. The default is 'grLe4ysf5HvZXuFKeZJlGZL7b'.

    Returns
    -------
    client : TYPE
        DESCRIPTION.

    """
    client = Socrata(domain,token)
    return client

def consultar_conjunto_paginado(id_conjunto, limit, ordenar, offset, **kwards):
    """
    

    Parameters
    ----------
    id_conjunto : TYPE
        DESCRIPTION.
    limit : TYPE
        DESCRIPTION.
    ordenar : TYPE
        DESCRIPTION.
    offset : TYPE
        DESCRIPTION.
    **kwards : TYPE
        DESCRIPTION.

    Returns
    -------
    None.

    """
    where = kwards.get('where', '')
    client = cliente_socrata()
    metadatos = consultar_metadata(id_conjunto)
    results = client.get(id_conjunto, limit=limit, order=ordenar, offset=limit*offset, where =where)
    results_df = pd.DataFrame.from_records(results)#
    results_df.reindex(columns=metadatos)
    results_df.to_parquet(f"../INPUTS/Conjunto_{id_conjunto}_{offset}.parquet.gzip")

    
def consultar_metadata(id_conjunto):
    """
    

    Parameters
    ----------
    id_conjunto : TYPE
        DESCRIPTION.

    Returns
    -------
    columns : TYPE
        DESCRIPTION.

    """
    client = cliente_socrata()
    metadatos = client.get_metadata(id_conjunto)
    columns = [x["fieldName"] for x in metadatos["columns"]]
    return columns


## Descargar información

In [29]:
# Carpetas del proyecto

try:
    os.mkdir(os.path.join(dir_proyecto,'INPUTS'))
except:
    None

try:
    shutil.rmtree(os.path.join(dir_proyecto,'INPUTS'))
except:
    None
os.mkdir(os.path.join(dir_proyecto,'INPUTS'))


In [30]:
# parámetros de las funciones
id_conjunto = 'jbjy-vk9h'
limit = 5000
ordenar = "id_contrato"
# filtro sobre el conjunto. Año 2023 cuyo valor sea superior a cien millones de pesos
where = "fecha_de_firma>='2023-01-01T00:00:00.000' and valor_del_contrato >= 100000000"


In [31]:
tamanio_conjunto = dimensionar_conjunto(id_conjunto, where=where)
tamanio_conjunto

48359

In [ ]:
loops = math.ceil(tamanio_conjunto/limit)

for offset in range (loops):
    consultar_conjunto_paginado(id_conjunto, limit, ordenar, offset, where = where)



## Probar la lectura de los datos


In [ ]:
contratacion = leer_parquet(os.path.join(dir_proyecto,'INPUTS'))
contratacion